# Introduction to DICOM & Voxels
![image.png](https://www.andersondiagnostics.com/wp-content/uploads/2017/09/CT-Scanner.jpg)

### What is a DICOM?
**D**igital **I**maging and **Co**mmunications in **M**edicine (DICOM) - an international standard related to the exchange, storage and communication of digital medical images. Prior to this format, there was no standardized way to transfer medical scans. So loading up a single patient's study outside the hospital, in older formats took about 10-30 minutes for a single scan! 

While DICOM 16-bit images (with values ranging from -32768..32767), other 8-bit greyscale images store values 0 - 255. These value ranges in DICOM are useful, as they correlate with the [Hounsfield Scale](https://en.wikipedia.org/wiki/Hounsfield_scale). Each voxel can store a large amount of information.

NB: If you want to get right to the code & image example, click [here](#example)


### Table of Contents

* [Workflow of a Radiologist](#workflow)
* [Why subdural hematomas are tricky](#explanation)
* [Using the Subdural Window](#revelation)
* [Example of subdural in the dataset](#example)

<a id="workflow"></a>
# Workflow of a Radiologist

This website shows a typical workflow, thought process and professional approach that a radiologists takes, when given a task to detect any abnormalities on a CT scan of the brain. [radiopedia.org](https://radiopaedia.org/articles/ct-head-an-approach?lang=gb) There are some details that I will skip over, as they are dealing with 3D scans in real life. In our dataset, for each patient ID we have only one slice (2D) of the brain to make our diagnosis from. 

> To be as good as a radiologist, you have to start thinking like one.

The issue that I want to highlight is how important windows are in a **radiologist's workflow**. I see most people are only using the brain matter window, which is able to pick up most abnormalities, I will show later that this might cause one to miss some diagnoses. This is one of the tricks where my professor used to catch me with during quizes. So keep your eyes open and take notes if you wish!

There are at least **5 windows** that a radiologist goes through for each scan!

1. Brain Matter window : W:80 L:40
2. Blood/subdural window: W:130-300 L:50-100
3. Soft tissue window: W:350–400 L:20–60
4. Bone window: W:2800 L:600
5. Grey-white differentiation window: W:8 L:32 or W:40 L:40

------------------------------------------------------------

Think of a window as an instruction to the computer to highlight only voxels which filfill a specific value. 
L = window level or center
W = window width or range

*Example*:     
Brain Matter window  
L = 40   
W = 80    
Voxels displayed range from 0 to 80   
(  Lower limit = 40 - (80/2), upper limit = 40 + (80/2)  )    
Voxel values outside this range will be completely black or white.      
credits to @amelnozieres for the correct calculation.

So you begin to see here, if it is possible to store 64k different values in each voxel - then the brain matter window only reveals a tiny fraction of this information. Converting this information to a compressed format such as .png may cause some loss of information. Similar to how we may lose some precision by converting float64 to float16 dtype.

<a id="explanation"></a>
# Why subdural hematomas are tricky

These critters are tricky by nature. If you check their definition, they usually are right next to the skull, longish in shape and follows the curvature of the skull.
Honestly, I need to do a double take to spot these nasties sometimes in the tiny screens of the emergency department.

Exhibit 1: Arrow marking subdural hemorrhage
![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F3454874%2F340f0cfb56257ff428b9e70193017731%2Fsubdural_window.png?generation=1569863200163417&alt=media)

Exhibit 2: Right with subdural window
![](http://www.learningradiology.com/images/neuroimages/subduralx2.jpg)


<a id="example"></a>

# Example of subdural hematoma in dataset

Here I will get straight into the code, and show you an example.
Credits to [Marco](https://www.kaggle.com/marcovasquez/basic-eda-data-visualization), [Nanashi](https://www.kaggle.com/jesucristo/rsna-introduction-eda-models) & [Richard McKinley](https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing) for their inspiration and awesome coding. Please go and upvote their kernels if you haven't done so yet.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pydicom
import os

print('Loaded in libraries!')

In [ ]:
TRAIN_IMG_PATH = "../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/"
TEST_IMG_PATH = "../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/"
BASE_PATH = '/kaggle/input/rsna-intracranial-hemorrhage-detection/'
TRAIN_DIR = 'stage_1_train_images/'
TEST_DIR = 'stage_1_test_images/'

train = pd.read_csv("../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv")
sub = pd.read_csv("../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv")
train_images = os.listdir("../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/")
test_images = os.listdir("../input/rsna-intracranial-hemorrhage-detection/stage_1_test_images/")

def window_image(img, window_center,window_width, intercept, slope, rescale=True):

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    
    if rescale:
        # Extra rescaling to 0-1, not in the original notebook
        img = (img - img_min) / (img_max - img_min)
    
    return img
    
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

    
    
def view_images(images, title = '', aug = None):
    width = 5
    height = 2
    fig, axs = plt.subplots(height, width, figsize=(15,5))
    
    for im in range(0, height * width):
        data = pydicom.read_file(os.path.join(TRAIN_IMG_PATH,'ID_'+images[im]+ '.dcm'))
        image = data.pixel_array
        window_center , window_width, intercept, slope = get_windowing(data)
        image_windowed = window_image(image, window_center, window_width, intercept, slope)


        i = im // width
        j = im % width
        axs[i,j].imshow(image_windowed, cmap=plt.cm.bone) 
        axs[i,j].axis('off')
        
    plt.suptitle(title)
    plt.show()

In [ ]:
#split out information to type and patientID

train['type'] = train['ID'].str.split("_", n = 3, expand = True)[2]
train['PatientID'] = train['ID'].str.split("_", n = 3, expand = True)[1]
train['filename'] = train['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".png")

sub['filename'] = sub['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".png")
sub['type'] = sub['ID'].apply(lambda st: st.split('_')[2])


In [ ]:
view_images(train[(train['type'] == 'epidural') & (train['Label'] == 1)][:10].PatientID.values, title = 'Images with epidural')

In [ ]:
train[(train['type'] == 'subdural') & (train['Label'] == 1)][:10]

In [ ]:
###ID_6ef1c9a94 good subdural

case = os.path.join(TRAIN_IMG_PATH,'ID_9d9cc6b01.dcm')

data = pydicom.read_file(case)

#print(data)
window_center , window_width, intercept, slope = get_windowing(data)
img = pydicom.read_file(case).pixel_array

#displaying the image

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,1, sharex='col', figsize=(10,24), gridspec_kw={'hspace': 0.1, 'wspace': 0})

ax1.set_title('Default window')
im1 = ax1.imshow(img,  cmap=plt.cm.bone)

ax2.set_title('Brain window')
img2 = window_image(img, 40, 80, intercept, slope)
im2 = ax2.imshow(img2, cmap=plt.cm.bone)

ax3.set_title('Subdural window')
img3 = window_image(img, 80, 200, intercept, slope)
im3 = ax3.imshow(img3, cmap=plt.cm.bone)
ax3.annotate('', xy=(150, 380), xytext=(120, 430),
            arrowprops=dict(facecolor='red', shrink=0.05),
            )
ax3.annotate('', xy=(220, 430), xytext=(190, 500),
            arrowprops=dict(facecolor='red', shrink=0.05),
            )

ax4.set_title('Bone window')
img4 = window_image(img, 600, 2800, intercept, slope)
im4 = plt.imshow(img4, cmap=plt.cm.bone)

for ax in fig.axes:
    ax.axis("off")
    
plt.show()
print(case)

# Conclusion

I hope to have shown you how important it is to incorporate subdural windows if you are a data scientist who wants to be as good as a radiologist. The evidence is as above, it is for you to examine and do drop me questions if you have in the chat below.

I hope that this has been of value to you on Kaggle. Perhaps this may be the final bit to tweak your model, or adds a boost to your LB score. 
This is my medical knowledge that is passed down to you, this is a way for me to give back to this awesome community from which I have learnt so much Python and coding from. 
Please upvote this kernel if you have found it helpful!

Credits and References

Kuzmak, P. M., Dayhoff, R. E., Gavrilov, S., Cebelinski, G., Shovestul, M. L., & Casertano, A. (2012). Streamlining importation of outside prior DICOM studies into an imaging system. Journal of digital imaging, 25(1), 70–77. doi:10.1007/s10278-011-9406-x

Heit, J. J., Iv, M., & Wintermark, M. (2017). Imaging of Intracranial Hemorrhage. Journal of stroke, 19(1), 11–27. doi:10.5853/jos.2016.00563

Dr Daniel J Bell and Dr Henry Knipe et al. Subdural window. Radiopedia.org

https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing

https://gist.github.com/lebedov/e81bd36f66ea1ab60a1ce890b07a6229

A side note about bone windows: useful to detect any bone fractures, which increases the probability of any hemorrhage in the brain.
I'm not sure how to code this into the algorithm, but I hope someone can enlighten me on this.